Questions
1. What's the default tax function?
2. How to calculate UBI's effect on income?

MU of consumption = disutility of labor
MU equates over periods

We have:
* params of ETR function
* baseline n and b
* UBI is only thing changing ETR

Multiply the change by the factor to see how it compares.

* omega is fraction by age
* lambda by skill bin

Default param ranges in json

In [41]:
import pandas as pd
import numpy as np
import ogusa
import plotly.express as px
import os
import distributed
import multiprocessing

In [57]:
base = pd.read_pickle('OUTPUT_BASELINE/SS/SS_vars.pkl')
reform = pd.read_pickle('OUTPUT_REFORM/SS/SS_vars.pkl')
reform_params = pd.read_pickle('OUTPUT_REFORM/model_params.pkl')

Change in labor supply: `nssmat` difference, collapse across ages.

```
def net_taxes(r, w, b, n, bq, factor, tr, theta, t, j, shift, method,
              e, etr_params, p):
```

In [24]:
assert np.allclose(base['rss'], reform['rss'])
assert np.allclose(base['wss'], reform['wss'])
assert np.allclose(base['factor_ss'], reform['factor_ss'])

In [29]:
# r, w, and factor are the same between base and reform.
# Label others as _base or _reform depending on what's
# needed for the UBI calculation.
r = base['rss']
w = base['wss']
# Label other variables as base.
# _s indicates for age s, vs. _splus1.
b_s_base = base['bssmat_s']  # Savings.
n_base = base['nssmat']  # Labor supply.
bq_base = base['bqssmat']  # Bequests received.
factor = base['factor_ss']  # Scaling factor from units to dollars.
tr_base = base['TR_ss']  # Government transfers to the household.
theta_base = base['theta']  # Social security replacement rate value.


This is copied from the `run_ogusa_*.py` file.

In [47]:
p = ogusa.Specifications(baseline=True, time_path=False)

In [61]:
reform_params.etr_params[-1, :, :]

array([[ 2.16807283e-22,  7.09923653e-05,  1.18316015e-10,
         2.66748450e-05,  3.94577918e-01, -2.31878502e-01,
         2.95997032e-01, -3.15287590e-01,  2.38143066e-01,
         3.21400436e-01, -3.15287590e-01,  1.00000000e+00],
       [ 1.16328687e-22,  7.08602724e-05,  2.17078901e-10,
         9.24413303e-18,  3.73043812e-01, -2.64976589e-01,
         1.71680804e-02, -3.19607022e-01,  2.71356793e-01,
         3.22974773e-01, -3.19607022e-01,  9.90907495e-01],
       [ 2.76618054e-10,  6.20012893e-05,  2.30792427e-23,
         1.08637151e-08,  8.00000000e-01, -3.15754552e-01,
         8.00000000e-01, -3.15754552e-01,  3.26912098e-01,
         3.26912098e-01, -3.15754552e-01,  8.60594844e-01],
       [ 8.41036712e-10,  4.07522546e-05,  3.38670660e-23,
         9.42016678e-09,  8.00000000e-01, -3.16065517e-01,
         8.00000000e-01, -3.22867489e-01,  3.27226172e-01,
         3.34096164e-01, -3.22867489e-01,  8.58749667e-01],
       [ 5.75774933e-23,  5.87149517e-05,  2.8150735

In [67]:
b_s_base.shape

(80, 7)

In [68]:
n_base.shape

(80, 7)

In [72]:
r * b_s_base + w * n_base * p.e #.shape

array([[0.18501207, 0.11210508, 0.11318395, 0.13045544, 0.16156734,
        0.26797914, 0.6714108 ],
       [0.17929378, 0.11748347, 0.12401859, 0.1449218 , 0.1792005 ,
        0.28404022, 0.72131939],
       [0.1938584 , 0.1370046 , 0.15063277, 0.17829   , 0.22018204,
        0.33542504, 0.85955514],
       [0.18259575, 0.13836818, 0.15777147, 0.18895491, 0.23316828,
        0.34367958, 0.89180468],
       [0.18575358, 0.15029473, 0.17711234, 0.21442302, 0.26440403,
        0.3790937 , 0.99399675],
       [0.18709881, 0.16096518, 0.19538014, 0.23884534, 0.29426434,
        0.41227132, 1.08961329],
       [0.18735782, 0.17074505, 0.21281771, 0.26242981, 0.32298511,
        0.44395012, 1.17868367],
       [0.1883307 , 0.18113419, 0.23118491, 0.28732406, 0.3532726 ,
        0.4783956 , 1.27494329],
       [0.18922695, 0.19140826, 0.24953116, 0.31235649, 0.38372083,
        0.51393925, 1.3754733 ],
       [0.186827  , 0.19803696, 0.263046  , 0.33141211, 0.4068152 ,
        0.54080273, 1.4

In [63]:
j = 0  # Try a list later to get all lifetime skill levels.
net_tax_ubi = ogusa.tax.net_taxes(
    r=r, w=w, b=b_s_base, n=n_base, bq=bq_base, factor=factor,
    tr=tr_base, theta=theta_base,
    t=None,
    j=j,
    shift=False,
    method='SS',
    e=p.e[:, j],
    etr_params=reform_params.etr_params[-1, :, :],
    p=p)

ValueError: operands could not be broadcast together with shapes (80,) (80,7) 

In [12]:
base.keys()

dict_keys(['Kss', 'K_f_ss', 'K_d_ss', 'Bss', 'Lss', 'Css', 'Iss', 'Iss_total', 'I_d_ss', 'nssmat', 'Yss', 'Dss', 'D_f_ss', 'D_d_ss', 'wss', 'rss', 'r_gov_ss', 'r_hh_ss', 'theta', 'BQss', 'factor_ss', 'bssmat_s', 'cssmat', 'bssmat_splus1', 'yss_before_tax_mat', 'bqssmat', 'TR_ss', 'trssmat', 'Gss', 'total_tax_revenue', 'business_tax_revenue', 'iit_payroll_tax_revenue', 'iit_revenue', 'payroll_tax_revenue', 'agg_pension_outlays', 'bequest_tax_revenue', 'wealth_tax_revenue', 'cons_tax_revenue', 'euler_savings', 'debt_service_f', 'new_borrowing_f', 'debt_service', 'new_borrowing', 'euler_labor_leisure', 'resource_constraint_error', 'etr_ss', 'mtrx_ss', 'mtry_ss'])

In [5]:
ogusa.tax.net_taxes()

<function ogusa.tax.net_taxes(r, w, b, n, bq, factor, tr, theta, t, j, shift, method, e, etr_params, p)>

Change between base and reform to `yss_before_tax` matches change to `nss` in the first period,
diverges after that.

In [79]:
assert np.allclose(reform['yss_before_tax_mat'][0, :] /
                   base['yss_before_tax_mat'][0, :],
                   reform['nssmat'][0, :] /
                   base['nssmat'][0, :])

In [80]:
assert not np.allclose(reform['yss_before_tax_mat'][1:, :] /
                       base['yss_before_tax_mat'][1:, :],
                       reform['nssmat'][1:, :] /
                       base['nssmat'][1:, :])

Show differences in labor supply.

In [102]:
nss_diff = pd.DataFrame(reform['nssmat'] / base['nssmat'] - 1)\
    .reset_index().melt('index')
nss_diff.columns = ['age', 'skill', 'n_diff']
nss_diff['n_diff_pct'] = (nss_diff.n_diff * 100).round(2)
nss_diff.skill = nss_diff.skill.astype(int)
# Age starts at 21.
nss_diff.age += 21

In [106]:
fig = px.line(nss_diff, x='age', y='n_diff_pct', color='skill',
              color_discrete_sequence=px.colors.sequential.Viridis)
fig.update_layout(
        title='Change to labor supply from $1,000 UBI in small open economy',
        xaxis_title='Age',
        yaxis_title='Change to labor supply',
        yaxis_ticksuffix='%',
        font=dict(family='Roboto'),
        hovermode='x',
        plot_bgcolor='white',
        legend_title_text='Lifetime income group'
    )
fig.update_traces(hovertemplate=None)